
# 🧩 06 — Agentic Patterns & Tools (Option A)

This notebook is your **pattern library** for designing and implementing **Agentic AI systems**.

It focuses on:

- Core **agentic patterns** (ReAct, Reflexion, Manager–Worker, Debate, etc.)
- How to shape **tools and actions** so agents use them safely and effectively
- Design patterns for **RAG agents**, **MCP-enabled agents**, and **multi-agent systems**
- Practical **do/don’t** examples and checklists

You can treat this as:

- A **design manual** for agent behaviors
- A **review checklist** for agent/tool design
- A bridge between **concepts (03)** and **code (04/05)** notebooks



## 1. Foundations: How to Think in Patterns

Before we name specific patterns, it helps to lock in a few meta-principles:

### 1.1 Agents Are Loops, Not Single Calls

Bad mental model:

> “An agent is just an LLM with a long prompt.”

Better mental model:

> “An agent is a **loop** that:  
> think → decide → act → observe → update → repeat (until done).”

This implies:

- Agents **accumulate state** (memory, observations)
- Actions are **discrete steps**, not just “one big answer”
- Tools must be designed for **iteration**, not one-shot miracles

### 1.2 Patterns Combine Roles, Tools, and Loops

Each pattern usually defines:

- **Roles** (planner, executor, critic, worker, etc.)
- **Tools** (retrieval, APIs, code execution, etc.)
- **Loop structure** (when to think, when to act, when to stop)

A good pattern:

- Is **easy to explain**
- Has **clear stopping conditions**
- Has **obvious “failure shapes”** you can test and monitor



## 2. ReAct Pattern (Reason + Act)

The ReAct pattern is the **canonical base** for agentic workflows.

### 2.1 Core Idea

Each step:

1. **THOUGHT** – model reasons about the situation
2. **ACTION** – picks a tool (or NONE)
3. **ACTION_INPUT** – arguments for the tool
4. **OBSERVATION** – tool result fed back to the next step

Repeat until:

- A **final answer** is ready, or  
- The agent hits a **step limit** or safety trigger  

### 2.2 When to Use

- You need a **single agent** that:
  - calls tools
  - explores multiple steps
  - uses intermediate observations

Good for:

- Simple research agents  
- Data lookup + synthesis  
- Short multi-step workflows  

### 2.3 Pitfalls

- Infinite loops if no stop condition  
- Overuse of tools if prompt doesn’t discourage unnecessary calls  
- Harder to debug if logs are not structured  

### 2.4 Best Practices

- Make the step format **explicit** (THOUGHT / ACTION / ACTION_INPUT / OBSERVATION)
- Log every step in a **structured way**
- Set a **max_steps** (e.g., 5–10)
- Clearly tell the model when to use `ACTION: NONE`



## 3. Reflexion Pattern (Self-Reflection & Retry)

Reflexion adds **self-critique** on top of ReAct.

### 3.1 Basic Flow

1. Agent produces an initial answer (maybe using tools)  
2. Agent (same or separate) evaluates the answer:
   - “Is this correct? Did I miss anything?”  
3. If needed, it:
   - plans a correction  
   - re-runs part of the process  
   - updates the final answer  

### 3.2 When to Use

- High-stakes answers
- Complex reasoning tasks
- RAG scenarios where **coverage and correctness** matter

### 3.3 Reflexion Prompts

Examples of self-eval prompts:

- “Critique the above answer. List possible mistakes or missing details.”
- “Given your critique, refine your answer.”

### 3.4 Risks

- Longer runtimes, more tokens  
- Self-critique can still hallucinate  
- Must have clear **retry limits**  

### 3.5 Pattern Variants

- Single-agent reflexion: agent critiques itself  
- Dual-agent reflexion: critic is a separate agent  
- External judge: LLM-as-judge or custom eval pipeline



## 4. Manager–Worker Pattern

One of the most practical and intuitive patterns.

### 4.1 Roles

- **Manager**:
  - understands user goals
  - decomposes work
  - delegates tasks
  - aggregates results  

- **Workers**:
  - specialized in certain tasks:
    - research
    - coding
    - analysis
    - retrieval
    - summarization  

### 4.2 Flow

1. Manager reads user request  
2. Manager breaks it into sub-tasks  
3. Manager assigns tasks to workers  
4. Workers produce outputs  
5. Manager composes final answer  

### 4.3 When to Use

- Multi-step tasks with distinct skills  
- You want clear separation of concerns  
- You plan to scale up with more worker types  

### 4.4 Implementation Tips

- Keep worker prompts **role-specific**  
- Give manager access to:
  - list of workers
  - their capabilities  
- Log which agents did what (for debugging & audit)



## 5. Debate Pattern

Two or more agents **argue** to refine an answer.

### 5.1 Flow

1. Agents produce initial answers  
2. Each critiques the other’s answer  
3. They iterate over:
   - argue → refine  
4. A judge (another agent or heuristic) picks a final answer  

### 5.2 Use Cases

- Ambiguous questions  
- Safety-sensitive contexts (e.g., risk analysis)  
- Ideation (brainstorm many angles)  

### 5.3 Cautions

- Higher cost  
- Can converge to **overconfident wrong answers** without a strong judge  
- Must prevent **toxic escalation** (moderation / safety filters)



## 6. Decomposition / Task-Tree Pattern

Agent explicitly builds a **task tree** or list of steps.

### 6.1 Example Prompt

- “Break the problem into 3–7 steps.”  
- “For each step, name: goal, tools needed, expected output.”

### 6.2 Flow

1. Decomposition step  
2. Execution per step (possibly with tools)  
3. Summary & aggregation  

### 6.3 Benefits

- Transparent plan  
- Can parallelize some steps  
- Useful for:
  - code generation
  - research
  - analysis projects  

### 6.4 Pitfalls

- Over-decomposing trivial tasks  
- Plan drift: actual steps diverge from initially stated plan  



## 7. Patterns for Tools & Actions

Patterns focused on **how tools are designed** for agents.

### 7.1 Narrow Tools vs “Do Everything” Tools

**Anti-pattern:**

- `do_anything(prompt: string)`  

**Pattern:**

- `search_docs(query, top_k)`  
- `get_user_profile(user_id)`  
- `list_open_tickets(assignee, limit)`  

### 7.2 ID-Based Flows

Make tools composable:

1. `search_*` → returns list with IDs  
2. `get_*_by_id` → returns full details  
3. `update_*_by_id` → optional for writes  

### 7.3 Human-Readable but Machine-Consistent

Tool names & descriptions should:

- help the LLM understand **when** to use them  
- be stable & consistent (no frequent renames)  

### 7.4 Observation Design

Observations should:

- be **structured JSON**, not just text  
- include:
  - success/error flags  
  - truncated markers  
  - metadata  

This makes logs + supervision easier.



## 8. Agentic RAG Patterns

Connecting to your **RAG Universe**.

### 8.1 Retrieval Planning

Instead of a single retrieve-then-answer:

1. Agent decides **what** to retrieve  
2. Calls RAG tools:
   - `search_knowledge_base`  
   - `get_document`  
3. Reflects: “Is this enough?”  
4. Possibly refines the query  

### 8.2 Common Agentic RAG Patterns

- **Query Rewriter Agent**:
  - Take user query → generate better search queries  

- **Context Selector Agent**:
  - Given many chunks, pick top-N relevant (+ diversify sources)  

- **Reasoning Agent with RAG**:
  - Merge retrieved info into multi-step reasoning  

### 8.3 Failure Modes & Fixes

- Over-retrieval → add **max chunks** & summarization step  
- Under-retrieval → reflexion + second retrieval round  
- Source confusion → ensure **citations** in responses  



## 9. MCP + Agents Patterns

Connecting to your **MCP Universe**.

### 9.1 MCP as Capability Layer

Instead of hardcoding tools:

- Agents see a catalog of MCP tools:
  - filesystem
  - HTTP
  - DB
  - RAG
  - DevOps
  - Finance  

### 9.2 Patterns

- **Capability Discovery Pattern**
  - Agent lists available tools (MCP) and decides which to use  

- **Guarded Action Pattern**
  - High-impact tools (deploy, rollback, write) behind extra checks  

- **MCP + Multi-Agent**
  - Manager decides which MCP tools to expose to which workers  

### 9.3 Safety Checklists (MCP + Agents)

- Limit which MCP tools are exposed to which agents  
- Add role-based prompts:
  - “You are only allowed to read logs, never deploy.”  
- Track:
  - tool usage logs
  - error rates
  - potentially harmful attempts  



## 10. Safety & Guardrail Patterns

### 10.1 Budget & Step Limits

- `max_steps` per run  
- `max_tokens` or cost budget  
- Hard stops on suspicious loops  

### 10.2 Tool Guardrails

- Validate inputs:
  - types
  - ranges
  - schemas  
- Deny dangerous operations:
  - shell exec
  - internal network scans
  - privileged writes  

### 10.3 Human-in-the-Loop

- For certain tools:
  - generate a **plan** first  
  - ask for human approval  
  - execute only after confirmation  

### 10.4 Red-Teaming Patterns

- Create adversarial prompts to test:
  - tool misuse
  - data exfiltration
  - over-permissioned behaviors  



## 11. Observability & Debugging Patterns

### 11.1 Structured Logging

Log per step:

- timestamp  
- agent ID  
- thought  
- action  
- action_input  
- observation  
- token usage (if available)  

### 11.2 Traces

- Group all steps for a single user query into a **trace**  
- Visualize as:
  - timeline
  - graph
  - tree  

### 11.3 Replay

- Re-run an agent trace with:
  - fixed LLM temperature
  - different prompts  
- Compare behaviors for regression testing  

### 11.4 Metrics

- tasks completed
- failures by type
- average steps per task
- tool usage distributions



## 12. Quick Pattern Reference

You can treat this list as a **mini index** of this notebook:

- ReAct Pattern  
- Reflexion Pattern  
- Manager–Worker Pattern  
- Debate Pattern  
- Decomposition / Task-Tree Pattern  
- ID-Based Tool Flows  
- Retrieval Planning (Agentic RAG)  
- Query Rewriter Agent  
- Context Selector Agent  
- MCP Capability Discovery Pattern  
- Guarded Action Pattern  
- Human-in-the-Loop Pattern  
- Budget & Step-Limit Guardrails  
- Structured Logging & Tracing Patterns  

When designing a new agent:

1. Pick 1–2 core patterns  
2. Decide which tools it can access  
3. Define clear stop conditions  
4. Add observability from the beginning  
5. Plan for at least one **safety pattern**
